### Computer Graphics: Perspective transformation
###### by Hamed Shah-hosseini
Explanation at: https://www.pinterest.com/HamedShahHosseini/
<br>https://github.com/ostad-ai/Computer-Graphics

In [6]:
from matplotlib import pyplot as plt
import numpy as np
from math import sin,cos,pi,tan
from ipywidgets import interact

In [7]:
#perspective projection for github
#parameters for the projection
width,height=640,480
aspect=width/height
near,far=1.,100.
fovy=30.
#vertices of a cube
vertex_list=np.array([[-1.,-1.,-1.],
                     [-1.,1.,-1.],
                      [1.,1.,-1.],
                      [1.,-1.,-1.],
                      [-1.,-1.,1.],
                      [-1.,1.,1.],
                      [1.,1.,1.],
                      [1.,-1.,1.]
                     ])
vertex_list[:,2]+=-7 #decrease z values

#edges of the cube
edge_list=np.array([[0,1],[1,2],[2,3],[3,0],
                    [0,4],[1,5],[2,6],[3,7],
                    [4,5],[5,6],[6,7],[7,4]])

def to_homogeneous(x,y,z):
    return np.array([x,y,z,np.ones_like(x)])

#fovY is in degrees
def perspective(fovY,aspect,zNear,zFar):
    epsilon=.01; fovY*=pi/360.
    if fovY<epsilon: fovY=epsilon
    f=1./tan(fovY)
    return np.array([[f/aspect,0.,0.,0.],
                     [0.,f,0.,0.],
                     [0.,0.,(zNear+zFar)/(zNear-zFar),
                      2.*zNear*zFar/(zNear-zFar)],
                     [0.,0.,-1.,0.]
                    ])

# matrix for rotation around y
def matrix_rotY(teta):
    c=cos(teta); s=sin(teta)
    return np.array([[c,0.,s,0.],
                     [0,1.,0.,0.],
                     [-s,0.,c,0.],
                     [0.,0.,0.,1.]])        

# matrix for rotation around x
def matrix_rotX(teta):
    c=cos(teta); s=sin(teta)
    return np.array([[1.,0.,0.,0.],
                     [0,c,-s,0.],
                     [0.,s,c,0.],
                     [0.,0.,0.,1.]])        

# matrix for rotation around z
def matrix_rotZ(teta):
    c=cos(teta); s=sin(teta)
    return np.array([[c,-s,0.,0.],
                     [s,c,0.,0.],
                     [0.,0.,1.,0.],
                     [0.,0.,0.,1.]])        

# rotation around axis x,y,z             
def rotateXYZ(tx,ty,tz,center=True):
    matXYZ=np.dot(matrix_rotX(tx),matrix_rotY(ty))
    matXYZ=np.dot(matXYZ,matrix_rotZ(tz))
    if center:
        xyz_mean=vertex_list.mean(axis=0)
        return xyz_mean+matrix_vector(matXYZ,
                                vertex_list-xyz_mean)
    else:
        return matrix_vector(matXYZ,vertex_list)
    
# project 3D vector into new space with the matrix    
def matrix_vector(matrix,vec3):
    temp=np.dot(matrix,to_homogeneous(vec3[:,0],vec3[:,1],vec3[:,2]))
    xs,ys,zs=temp[0]/temp[3],temp[1]/temp[3],temp[2]/temp[3]
    return np.vstack([xs,ys,zs]).T   

#compute perspective matrix
matrix_pers=perspective(fovy,aspect,near,far)

# tx,ty,tz are angles of rotation in radians
def run(tx,ty,tz):
    xyz=rotateXYZ(tx,ty,tz)
    xyz=matrix_vector(matrix_pers,xyz)
    #scaling into width and height 
    x_vp,y_vp=.5*width*(xyz[:,0]+1),.5*height*(xyz[:,1]+1)
    #compute pseudo-brightness from depth
    brightness=.5*(1+xyz[:,2])
    plt.scatter(x_vp,y_vp,c=brightness,s=50,zorder=3)
    for edge in edge_list:
        col=.5*(brightness[edge[0]]+brightness[edge[1]])
        plt.plot([x_vp[edge[0]],x_vp[edge[1]]],
                  [y_vp[edge[0]],y_vp[edge[1]]],'-',
                 color=[col,col**4,col**3])
    plt.xlim([0,width]); plt.ylim([0,height])
    plt.axis('off'); plt.show()
interact(run,tx=(-3.1,3.14,.1),ty=(-3.1,3.14,.1),tz=(-3.1,3.14,.1))

interactive(children=(FloatSlider(value=0.0, description='tx', max=3.14, min=-3.1), FloatSlider(value=0.0, des…

<function __main__.run(tx, ty, tz)>